In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib

In [ ]:
import numpy as np
from pathlib import Path
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
import os
from qgsw.utils.sorting import sort_files
from mpl_toolkits.axes_grid1 import make_axes_locatable
from qgsw.run_summary import RunSummary

In [ ]:
ROOT = Path(os.path.abspath('')).parent

In [ ]:
folder = ROOT.joinpath("output/g5k/imports")
run = RunSummary.from_file(folder.joinpath("_summary.toml"))

In [ ]:
steps, files = sort_files(folder.rglob("*.npz"), prefix=run.configuration.model.prefix, suffix = ".npz")

In [ ]:
file = files[80]
fields_map = {
    "vorticity": {"key": "omega", "label":"Vorticity"},
    "pressure": {"key": "p", "label":"Pressure"},
}
field = fields_map["vorticity"]

def mean_threshold(top:np.ndarray, bottom:np.ndarray) -> tuple[float, float]:
    mean_top = np.mean(np.abs(top[np.abs(top) > 1e-5 ]))
    std_top = np.std(np.abs(top[np.abs(top) > 1e-5 ]))
    mean_bottom = np.mean(np.abs(bottom[np.abs(bottom) > 1e-5 ]))
    std_bottom = np.std(np.abs(bottom[np.abs(bottom) > 1e-5 ]))
    return mean_top + 1*std_top , mean_bottom + 1*std_bottom

def quantile_threshold(top: np.ndarray, bottom:np.ndarray) -> tuple[float, float]:
    quantile_value = 0.95
    abs_top = np.abs(top)[np.abs(top) > 1e-5]
    top_quantile = np.quantile(abs_top, quantile_value)
    abs_bottom = np.abs(bottom)[np.abs(bottom) > 1e-5]
    bottom_quantile = np.quantile(abs_bottom, quantile_value)
    return top_quantile, bottom_quantile


def get_thresholds(top:np.ndarray, bottom:np.ndarray) -> tuple[float, float]:
    return mean_threshold(top, bottom)

In [ ]:
# %matplotlib inline
plot = False
if plot:
    plt.ion()
    cmap = plt.cm.bwr
    fig, axes = plt.subplots(3, 2, figsize=(18,12))
    fig.subplots_adjust(right=0.85)

    cbars = np.empty_like(axes)

    cbars[0,0] = make_axes_locatable(axes[0,0]).append_axes('right', size='5%', pad=0.05)
    cbars[0,1] = make_axes_locatable(axes[0,1]).append_axes('right', size='5%', pad=0.05)
    cbars[1,0] = make_axes_locatable(axes[1,0]).append_axes('right', size='5%', pad=0.05)
    cbars[1,1] = make_axes_locatable(axes[1,1]).append_axes('right', size='5%', pad=0.05)
    cbars[2,0] = make_axes_locatable(axes[2,0]).append_axes('right', size='5%', pad=0.05)

offset = 1
last = -1
means = [0] * offset
stds = [0] * offset
for i in range(len(files[offset:last])+1):
    file = files[i+offset]
    step = steps[i+offset]
    data = np.load(file)[field["key"]]
    top = data[0,0]
    bottom = data[0,1]
    top_threshold, bottom_threshold = get_thresholds(top, bottom)
    top_above_threshold = np.abs(top) > top_threshold
    bottom_above_threshold = np.abs(bottom) > bottom_threshold

    if plot:
        fig.suptitle(f"Step: {step} / {steps[last]}")

        axes[0,0].cla()
        axes[0,1].cla()
        axes[1,0].cla()
        axes[1,1].cla()
        axes[2,0].cla()
        axes[2,1].cla()

        cbar = axes[0,0].imshow(top, cmap=cmap, vmin=-np.max(np.abs(top)), vmax=np.max(np.abs(top)))
        fig.colorbar(cbar,cax=cbars[0,0])
        axes[0,0].set_title(f"{field['label']} - Surface")

        cbar = axes[0,1].imshow(bottom, cmap=cmap, vmin=-np.max(np.abs(bottom)), vmax=np.max(np.abs(bottom)))
        fig.colorbar(cbar,cax=cbars[0,1])
        axes[0,1].set_title(f"{field['label']} - Bottom")

        cbar = axes[1,0].imshow(top_above_threshold, cmap=plt.cm.binary)
        fig.colorbar(cbar,cax=cbars[1,0])
        axes[1,0].set_title(f"{field['label']} above threshold - Surface")

        cbar = axes[1,1].imshow(bottom_above_threshold, cmap=plt.cm.binary)
        fig.colorbar(cbar,cax=cbars[1,1])
        axes[1,1].set_title(f"{field['label']} above threshold - Bottom")

    ratio = np.zeros(top.shape)
    below_quantiles = top_above_threshold & bottom_above_threshold
    ratio[below_quantiles] = bottom[below_quantiles] / top[below_quantiles]

    if plot:
        cbar = axes[2,0].imshow(ratio, cmap=cmap, vmin=-np.max(np.abs(ratio)), vmax=np.max(np.abs(ratio)))
        fig.colorbar(cbar,cax=cbars[2,0])
        axes[2,0].set_title(f"{field['label']} ratio - Bottom / Surface")

    means.append(np.mean(ratio[below_quantiles]))
    stds.append(np.std(ratio[below_quantiles]))
    if plot:
        axes[2,1].set_xlim([steps[0], steps[last]])
        # axes[2,1].set_ylim([-1, 1])
        axes[2,1].plot(steps[:i+offset+1], means)
        axes[2,1].fill_between(steps[:offset+i+1] ,np.array(means) - np.array(stds), np.array(means) + np.array(stds), color='grey', alpha=0.2)
        axes[2,1].set_title(f"Mean Ratio Evolution Over Frames")
        plt.pause(0.01)
if plot:
    plt.ioff()

In [ ]:
steps_total = range(steps[-1]+1)
# mean_step_map = {steps[i]:means[i] for i in range(len(steps))}
# means_total = [mean_step_map[step] if step in steps else np.nan for step in steps_total]
means_total = np.interp(steps_total, steps, means)

In [ ]:
plt.plot(steps_total, means_total)
plt.show()

In [ ]:
kernel_size = 1e5
sigma = 0.05
X = np.linspace(-1, 1, int(kernel_size))

In [ ]:
kernel = 1/(sigma*np.sqrt(2*np.pi))*np.exp(-np.power(X,2)/(2*sigma))
kernel = kernel /np.sum(kernel)

In [ ]:
plt.plot(X,kernel)
plt.show()

In [ ]:
filtered = np.convolve(np.pad(means_total, (len(kernel)-1,len(kernel)-1), mode="edge"),kernel, mode="same")
plt.plot(steps_total,filtered[len(kernel)-1:-(len(kernel)-1)], label="Filtered Points")
plt.scatter(steps, means, c='red', alpha=0.5, label = "Reference Points")
plt.legend()
plt.show()